# Basic Counting of musical traces

(old notebook from november 2024)

We define multiple functions to count multiple traces, namely :
- Jugglable musics of length n, with max height h, and b different balls
- Jugglable musics of length n, with max height h, and b identical balls

For each of those, we can add a periodicity condition, and delete cyclic shifts

All follow Vanilla Siteswap for now.
Later experiment with :
- multiplex with k balls in hand at max
- mixing different and identical balls

Another way to proceed would be to construct automaton

Note : for b balls, be cautious of the first b/h elements of the sequence : it is possible a note will never occur.

In [2]:
from itertools import product

In [ ]:
def insert(L: list[int], new_elem: int):
    """Inserts 

    Args:
        l (list[int]): _description_
        new_elem (int): _description_
    """
    for i in range(len(L) - 1):
        L[i] = L[i+1]
    L[-1] = new_elem

def iter_identical(n: int, h: int, b:int=1):
    for seq in product([0, 1], repeat=n):
        last_ones_occurences = [-1]*b
        for i, elem in enumerate(seq):
            if elem == 1:
                insert(last_ones_occurences, i)
            if i - last_ones_occurences[0] >= h:
                break
        else:
            yield seq
        
def iter_different(n: int, h: int, b:int=1):
    for seq in product(range(b+1), repeat=n):
        last_balls_occurences = [0] + [-1]*b
        for i, elem in enumerate(seq):
            if elem != 0:
                if i - last_balls_occurences[elem] > h:
                    break
                last_balls_occurences[elem] = i
        else:
            # Additional check at the end
            for i in range(1, len(last_balls_occurences)):
                if n - last_balls_occurences[i] > h:
                    break
            else:
                yield seq

list(iter_different(3, 2, 2))

# def iter_different(n: int, h: int):
#     for seq in product([0, 1], repeat=n):
#         last_one = -1
#         for i, elem in enumerate(seq):
#             if elem == 1:
#                 last_one = i
#             if i - last_one >= h:
#                 break
#         else:
#             yield seq

[(1, 2, 1), (2, 1, 2)]

In [76]:
list(iter_different(5, 3, 3))

[(1, 2, 3, 1, 2),
 (1, 3, 2, 1, 3),
 (2, 1, 3, 2, 1),
 (2, 3, 1, 2, 3),
 (3, 1, 2, 3, 1),
 (3, 2, 1, 3, 2)]

In [80]:
for n in range(20):
    print(len(list(iter_different(n, 5, 2))))

1
3
9
27
81
180
480
1268
3320
8636
22448
58628
152944
398788


KeyboardInterrupt: 

In [ ]:
list(iter_identical(n, 3, 2))

In [ ]:
from typing import Generic, Iterable, TypeVar

T = TypeVar("T")

class Test(Generic[T]):
    def __init__(self, iterable: Iterable[T]):
        
        

In [ ]:

from itertools import product
from typing import Any, Callable, Iterable, ParamSpec, TypeVar

def insert(l: Iterable[int], new_elem: int):
    for i in range(len(l) - 1):
        l[i] = l[i+1]
    l[-1] = new_elem

def iter_identical(n: int, h: int, b:int=1, periodic=False):
    if periodic and n == 0:
        nb_it = 0
    elif periodic:
        nb_it = n + h
    else:
        nb_it = n
    for seq in product([0, 1], repeat=n):
        last_ones_occurences = [-1]*b
        for i in range(nb_it):
            elem = seq[i % n]
            if elem == 1:
                insert(last_ones_occurences, i)
            if i - last_ones_occurences[0] >= h:
                break
        else:
            yield seq
        
def iter_different(n: int, h: int, b:int=1, periodic:bool=False):
    if periodic and n == 0:
        nb_it = 0
    elif periodic:
        nb_it = n + h
    else:
        nb_it = n
    for seq in product(range(b+1), repeat=n):
        # Periodic or not, check all balls are in the beginning
        present = [False]*(b+1)
        present[0] = True
        for i in range((min(h, n))):
            present[seq[i]] = True
        if not all(present):
            continue

        # If not periodic, also check at the end
        # (if periodic, redundant with previous check)
        if not periodic:
            present = [False]*(b+1)
            present[0] = True
            for i in range(max(0, n-h), n):
                present[seq[i]] = True
            if not all(present):
                continue

        last_balls_occurences = [0] + [-1]*b
        for i in range(nb_it):
            elem = seq[i % n]
            if elem != 0:
                if i - last_balls_occurences[elem] > h:
                    break
                last_balls_occurences[elem] = i
        else:
            yield seq

def left_shift(l: list, idx: int):
        return tuple(l[(i + idx) % len(l)] for i in range(len(l)))

P = ParamSpec('P')
T = TypeVar('T')

# Décorateur pour supprimer des solutions qui sont des shifts cycliques d'une déjà trouvée.
def no_cyclic(func: Callable[P, Iterable[T]]) -> Callable[P, Iterable[T]]:
    def new_function(*args: P.args, **kwargs: P.kwargs):
        seen = set()
        for elem in func(*args, **kwargs):
            for i in range(1, len(elem)):
                if left_shift(elem, i) in seen:
                    break
            else:
                seen.add(tuple(elem))
                yield elem
    return new_function

In [2]:
@no_cyclic
def iter_different_no_cyclic(n: int, h: int, b:int=1):
    yield from iter_different(n, h, b, True)

@no_cyclic
def iter_identical_no_cyclic(n: int, h: int, b:int=1):
    yield from iter_identical(n, h, b, True)

In [4]:
h = 4
b = 3
for n in range(20):
    print(len(list(iter_different(n, h, b, True))))

0
0
0
6
60
0
24
126
372
132
600
1716
3324
3354
9744
21696


KeyboardInterrupt: 

In [6]:
print(list(iter_different(4, 4, 2, True)))
print(list(iter_different(5, 4, 2, True)))

[(0, 0, 1, 2), (0, 0, 2, 1), (0, 1, 0, 2), (0, 1, 1, 2), (0, 1, 2, 0), (0, 1, 2, 1), (0, 1, 2, 2), (0, 2, 0, 1), (0, 2, 1, 0), (0, 2, 1, 1), (0, 2, 1, 2), (0, 2, 2, 1), (1, 0, 0, 2), (1, 0, 1, 2), (1, 0, 2, 0), (1, 0, 2, 1), (1, 0, 2, 2), (1, 1, 0, 2), (1, 1, 1, 2), (1, 1, 2, 0), (1, 1, 2, 1), (1, 1, 2, 2), (1, 2, 0, 0), (1, 2, 0, 1), (1, 2, 0, 2), (1, 2, 1, 0), (1, 2, 1, 1), (1, 2, 1, 2), (1, 2, 2, 0), (1, 2, 2, 1), (1, 2, 2, 2), (2, 0, 0, 1), (2, 0, 1, 0), (2, 0, 1, 1), (2, 0, 1, 2), (2, 0, 2, 1), (2, 1, 0, 0), (2, 1, 0, 1), (2, 1, 0, 2), (2, 1, 1, 0), (2, 1, 1, 1), (2, 1, 1, 2), (2, 1, 2, 0), (2, 1, 2, 1), (2, 1, 2, 2), (2, 2, 0, 1), (2, 2, 1, 0), (2, 2, 1, 1), (2, 2, 1, 2), (2, 2, 2, 1)]
[(0, 1, 1, 2, 2), (0, 1, 2, 1, 2), (0, 1, 2, 2, 1), (0, 2, 1, 1, 2), (0, 2, 1, 2, 1), (0, 2, 2, 1, 1), (1, 0, 1, 2, 2), (1, 0, 2, 1, 2), (1, 0, 2, 2, 1), (1, 1, 0, 2, 2), (1, 1, 1, 2, 2), (1, 1, 2, 0, 2), (1, 1, 2, 1, 2), (1, 1, 2, 2, 0), (1, 1, 2, 2, 1), (1, 1, 2, 2, 2), (1, 2, 0, 1, 2), (1, 2, 0,

In [14]:

print(list(iter_identical(3, 3, 2, False)))

[(0, 1, 1), (1, 0, 1), (1, 1, 0), (1, 1, 1)]


In [13]:
h = 3
b = 2
periodic = False
for n in range(20):
    print(len(list(iter_identical(n, h, b, periodic))))

1
2
4
4
6
9
13
19
28
41
60
88
129
189
277
406
595
872
1278
1873
